In [ ]:
# read the data 
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = 10, 10
from utilities import *
import math
from sklearn.model_selection import train_test_split
import pandas as pd
from itertools import combinations

In [ ]:
def process_inhval(*args):
    for val in args: 
        if val == 'NA':
            yield 0
        else:
            yield float(val)

In [ ]:
readfilename = 'PI_DataSet_6_19_all.txt'
r = open(readfilename)
header = process_line(next(r))
inh_start = header.index('FPV') 
inh_end = header.index('DRV')+1
vec_start = header.index('count_vec_start')
vec_end = len(header)
vals = []
for line in r:
    if line!= '\n':
        line = process_line(line)
        token = list(process_inhval(*line[inh_start:inh_end])) + [int(x) for x in line[vec_start:vec_end]]
        vals.append(token)
r.close()

In [ ]:
vals = np.array(vals)

In [ ]:
ind_inh = {v:i for i,v in enumerate(header[inh_start:inh_end])}

In [ ]:
def filter_inhs(data, ind_dict, *args,classify=True):
    vec_length = 210
    names = sorted([(ind_dict[name],name) for i,name in enumerate(args)])
    inds = list(map(lambda x:x[0], names))
    names = list(map (lambda x:x[1], names))
    tmp = np.hstack((data[:,inds],data[:,8:]))
    df = pd.DataFrame(tmp, columns = ['c'+str(i) for i in range(len(tmp[0]))])
    query_string = ' & '.join(['c'+str(i)+">0" for i in range(len(inds))])
    df = df.query(query_string)
    if classify: 
        for i,name in enumerate(names):
            df[names[i]] = np.where(df['c'+str(i)] > 3.0 , '1', '0')
        df['target'] = df[names].apply(lambda x:int(''.join(x),2),axis = 1)
        return df.iloc[:,len(names):(-len(names)-1)].values,df['target'].values.reshape(-1,1)
    else:
        return df.iloc[:,len(names):].values, df.iloc[:,0:len(names)]

In [ ]:
X,y = filter_inhs(vals,ind_inh,'TPV', classify = False)

In [ ]:
max_ = X.max()
X = X/max_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
X_train.shape, y_train.shape